In [1]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit.providers.basic_provider import BasicSimulator

In [2]:
#Quantum Fourier Transform
# q_0 represents the inital qubit of the QFT
def QFT(q_circuit, n, q_0):
    q_circuit.barrier()
    for k in range(q_0, q_0 + n):
        q_circuit.h(k)
        for i in range(k + 1, q_0 + n):
            q_circuit.cp(np.pi / (2 ** (i - k)), k, i)
        q_circuit.barrier()
        
    #Swap of all of qubits
    for l in range(n//2):
        q_circuit.swap(q_0 + l, n - 1 - l + q_0)
    q_circuit.barrier()

In [3]:
#Quantum Fourier Transform Inverse
# q_0 represents the inital qubit of the QFT inverse
def QFT_I(q_circuit, n, q_0):
    #Swap of all of qubits
    for l in range(n//2):
        q_circuit.swap(l + q_0, n - 1 - l + q_0)
    q_circuit.barrier()
    
    for k in range(n):
        for i in range(k):
            q_circuit.cp(- np.pi / (2 ** (k - i)), n - k - 1 + q_0, n - i - 1 + q_0)
        q_circuit.h(n + q_0 - k - 1)
        q_circuit.barrier()

In [4]:
def sum_QFT(a, b, n):
    qc = QuantumCircuit(2 * n, n)
    
    # Inicialization of the state |a>
    for i in range(n):
        if np.binary_repr(a, n)[i] == '1':
            qc.x(i)

    # Inicialization of the state |b>
    for i in range(n):
        if np.binary_repr(b, n)[i] == '1':
            qc.x(i + n)
    
    QFT(qc, n, n) # Quantum Fourier Transform

    # Sum of the qubits in the Fourier Transform representation of the qubit |b>
    for k in range(n):
        for l in range(n):
            qc.cp(2 ** (n - 1 - k) * np.pi / 2 ** l, k, l + n)
        qc.barrier()
    
    QFT_I(qc, n, n) # Quantum Fourier Transform Inverse
    
    # Measure the las n qubits
    for k in range(n):
        qc.measure(k + n, n - 1- k)
        
    qc.draw('mpl')
    
    simulator_basic = BasicSimulator()
    
    compilado = transpile(qc, simulator_basic)
    
    job = simulator_basic.run(compilado, shots =  1)
    
    # Excecute the circuit
    result = job.result()

    # Return a + b
    return int(max(result.get_counts()), 2)

In [5]:
a = 2
b = 3
# n of qubits to represent a + b
n = 4

# Print the result of the sum of a and b
print(f'{a} + {b} = {sum_QFT(a, b, n)}')

2 + 3 = 5
